# Importing the dataset into a dataframe

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk


#import seaborn as sns
import sklearn.metrics as metrics
import sklearn.model_selection as ms
import sklearn.preprocessing as preprocess
from sklearn.model_selection import train_test_split


In [2]:
# !pip install pyarabic
# !pip install langdetect
# !pip install nltk

# from langdetect import detect
# import pyarabic.araby as araby
# nltk.download("all")


In [3]:
df = pd.read_csv('freqEncoded.csv')
df.drop(["text","stemmed_text","normalized","clean_text"],axis=1,inplace=True)

In [4]:
df.head()

,label,freq
0,1,"{'متز': 1, 'نوع': 1, 'ما': 1, 'نظف': 1, 'وقع':..."
1,1,"{'احد': 1, 'سبب': 1, 'نجح': 1, 'امر': 2, 'كل':..."
2,1,"{'هدف': 1, 'نقل': 1, 'صخب': 1, 'شرع': 1, 'قهر'..."
3,1,"{'خلص': 3, 'بدء': 1, 'الل': 1, 'بهر': 1, 'زي':..."
4,1,"{'ياس': 1, 'جزء': 2, 'لا': 2, 'دبي': 1, 'ندق':..."


# Unique Dictionary

In [5]:


def get_unique(df,colname):
  sentences_arr=df[colname].to_numpy() #convert df to np array
  words=' '.join(sentences_arr).split(' ') # join all strings into one string then splitting to get the words
  words=np.array(words) #convert to np array
  unique,counts = np.unique(words,return_counts=True)
  return unique, counts

def get_unique_dict(unique,counts):
  unique_dict=dict(zip(unique,counts))
  unique_dict=dict(sorted(unique_dict.items(), key=lambda item: item[1],reverse=True))
  return unique_dict
def remove_redundant_words(unique_dict):
  stop_words=['من','على','عن','في','فى','و','ان','هذا','او','كتب','...','.','','الى','فيه','انه','قبل','//','..','،',':',"؟",'/']
  for k, v in list(unique_dict.items()):
      if(unique_dict[k]<100 or k in stop_words):
        del unique_dict[k]

  return unique_dict



In [6]:
df_copy= pd.read_csv('freqEncoded.csv')
unique,counts=get_unique(df_copy,"normalized")
unique_dict=get_unique_dict(unique,counts)
unique_dict=remove_redundant_words(unique_dict)

In [7]:
unique_dict

{'لا': 46481,
 'روي': 39737,
 'ما': 35527,
 'جدا': 33678,
 'كان': 31659,
 'قرء': 30133,
 'كثر': 29226,
 'لم': 28285,
 'ندق': 28261,
 'عمل': 24589,
 'وجد': 24462,
 'كل': 24394,
 'غرف': 22749,
 'حدث': 21929,
 'وقع': 21515,
 'بعد': 20845,
 'غير': 20823,
 'علم': 20637,
 'جمل': 20497,
 'لكن': 20115,
 'التي': 19829,
 'نفس': 18656,
 'مع': 18473,
 'فكر': 18459,
 'الل': 18121,
 'شخص': 17767,
 'هو': 17455,
 'جمع': 17291,
 'بعض': 15945,
 'جيد': 15644,
 'خدم': 15180,
 'نظف': 14401,
 'كانت': 14269,
 'شكل': 14102,
 'عرف': 14049,
 'هذه': 13567,
 'بين': 13480,
 'شعر': 13331,
 'عند': 13288,
 'وحد': 13262,
 'الذي': 13228,
 'كلم': 13176,
 'ولا': 13069,
 'عجب': 12606,
 'قرب': 12439,
 'اخر': 12411,
 'الا': 12117,
 'حتى': 11898,
 'رءع': 11889,
 'قدم': 11874,
 'نظر': 11553,
 'علي': 11458,
 'نهي': 11402,
 'دخل': 11274,
 'سلم': 11237,
 'شيء': 11135,
 'سلب': 10727,
 'طرق': 10454,
 'ليس': 10309,
 'اول': 10136,
 'فصل': 10062,
 'سيء': 9866,
 'وقف': 9742,
 'عبر': 9653,
 'ذكر': 9345,
 'خلف': 9244,
 'ذلك': 9195,
 'حق

# More preprocessing

### Convert string dict to actual dict

In [8]:
import ast

#converting string dict to dict
df['freq']=df['freq'].apply(lambda x: ast.literal_eval(x))
print(type(df.head()["freq"][0]))

<class 'dict'>


### Add colum for each words whose value is the frequency of appearance in the sentence

In [9]:
def encode_freq(freq_dict):
    words=freq_dict.keys()
    arr= [freq_dict[word] if word in words else 0 for word in unique_dict.keys() ] 
    return np.array(arr)

In [ ]:
freq_columns=df['freq'].apply(encode_freq)
# dfOut=pd.DataFrame()
# dfOut['label']=df['label'] #copy labels
df[list(unique_dict.keys())]=pd.DataFrame(freq_columns.tolist()) #convert list of frequencies to columns

In [12]:
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 3683 entries, label to يمض
dtypes: int32(3681), int64(1), object(1)
memory usage: 1.4+ GB
None


,label,freq,لا,روي,ما,جدا,كان,قرء,كثر,لم,...,سلو,فتل,فقم,قمن,كيلو,مل،,هدل,هرد,وذك,يمض
0,1,"{'متز': 1, 'نوع': 1, 'ما': 1, 'نظف': 1, 'وقع':...",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,"{'احد': 1, 'سبب': 1, 'نجح': 1, 'امر': 2, 'كل':...",1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,"{'هدف': 1, 'نقل': 1, 'صخب': 1, 'شرع': 1, 'قهر'...",0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,"{'خلص': 3, 'بدء': 1, 'الل': 1, 'بهر': 1, 'زي':...",1,2,2,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,"{'ياس': 1, 'جزء': 2, 'لا': 2, 'دبي': 1, 'ندق':...",2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,-1,"{'عرفش': 1, 'ليه': 1, 'كنت': 1, 'كمل': 1, 'وهي...",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99995,-1,"{'لا': 3, 'سحق': 1, 'يكون': 1, 'كنق': 1, 'لنه'...",3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,-1,"{'ضعف': 1, 'جدا': 1, 'ولم': 1, 'متع': 1, 'به':...",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,-1,"{'ملة': 1, 'جدا': 1, 'حمد': 1, 'حسن': 1, 'علو'...",0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df.to_csv('freqEncoded.csv', encoding = 'utf-8-sig',index=False) 

In [14]:
data =df.copy()

# TF-IDF

In [15]:
#joined words into a sentence again to use sklearn model
df["text"]=df["freq"].apply(lambda freq_dict: ' '.join(list(freq_dict.keys())) )
df

,label,freq,text
0,1,"{'متز': 1, 'نوع': 1, 'ما': 1, 'نظف': 1, 'وقع':...",متز نوع ما نظف وقع جهز شاطيء طعم
1,1,"{'احد': 1, 'سبب': 1, 'نجح': 1, 'امر': 2, 'كل':...",احد سبب نجح امر كل شخص هذه دول عشق ترب نحن نحب...
2,1,"{'هدف': 1, 'نقل': 1, 'صخب': 1, 'شرع': 1, 'قهر'...",هدف نقل صخب شرع قهر هدء جبل شيش عرف حقق ما جرى...
3,1,"{'خلص': 3, 'بدء': 1, 'الل': 1, 'بهر': 1, 'زي':...",خلص بدء الل بهر زي فيل زرق حمد راد خطى رحل قرء...
4,1,"{'ياس': 1, 'جزء': 2, 'لا': 2, 'دبي': 1, 'ندق':...",ياس جزء لا دبي ندق كامل خدم ريح نفس وجد
...,...,...,...
99994,-1,"{'عرفش': 1, 'ليه': 1, 'كنت': 1, 'كمل': 1, 'وهي...",عرفش ليه كنت كمل وهي مش عجب حدث بطء ممل روي اط...
99995,-1,"{'لا': 3, 'سحق': 1, 'يكون': 1, 'كنق': 1, 'لنه'...",لا سحق يكون كنق لنه سيء شي وجد خدم فطر صبح ستي...
99996,-1,"{'ضعف': 1, 'جدا': 1, 'ولم': 1, 'متع': 1, 'به':...",ضعف جدا ولم متع به كل قصه سرد لحل شهد بدن فكر
99997,-1,"{'ملة': 1, 'جدا': 1, 'حمد': 1, 'حسن': 1, 'علو'...",ملة جدا حمد حسن علو فنن وصف عند دقق حد ثني قرء...


In [19]:
#Splitting data
X_train,X_test,Y_train, Y_test = train_test_split(df['text'], df['label'], test_size=0.25, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (74999,) (74999,) Test:  ((25000,), (25000,))


### TF*IDF Vectorizer

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

<74999x3593 sparse matrix of type '<class 'numpy.float64'>'
	with 2441943 stored elements in Compressed Sparse Row format>

In [72]:
tfidf_tokens = vectorizer.get_feature_names()
df_tfidf=pd.DataFrame(data=tf_x_train.toarray(),columns=tfidf_tokens)

In [91]:
print(tf_x_train.toarray())
df_tfidf

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


,ءءخذ,ءءخر,ءات,ءبعد,ءتمنى,ءتى,ءثث,ءثر,ءجب,ءجد,...,يني,يه,يهد,يهم,يوج,يوح,يود,يول,يوم,يون
0,0.0,0.0,0.0,0.0,0.0,0.15731,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74994,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74995,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74996,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74997,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#  Training Models



In [24]:
from sklearn.tree import DecisionTreeClassifier
DT_clf = DecisionTreeClassifier(random_state=0).fit(tf_x_train, Y_train)


In [25]:
DT_pred = DT_clf.predict(tf_x_test)
from sklearn.metrics import classification_report
print(classification_report(Y_test, DT_pred))

              precision    recall  f1-score   support

          -1       0.57      0.58      0.57      8244
           0       0.47      0.47      0.47      8376
           1       0.57      0.57      0.57      8380

    accuracy                           0.54     25000
   macro avg       0.54      0.54      0.54     25000
weighted avg       0.54      0.54      0.54     25000



In [28]:
from sklearn.tree import plot_tree
fig = plt.figure(figsize=(25,20))
plot_tree(DT_clf, max_depth=2, feature_names=tf_x_test.columns, class_names=["negative", "positive", "neutral"], filled=True);

AttributeError: columns not found

<Figure size 1800x1440 with 0 Axes>

## Naive Bayes

In [50]:
from sklearn.naive_bayes import GaussianNB
NV_clf = GaussianNB().fit(tf_x_train.toarray(), Y_train)
NV_pred = NV_clf.predict(tf_x_test.toarray())
print(classification_report(Y_test, NV_pred))

              precision    recall  f1-score   support

          -1       0.54      0.61      0.57      8244
           0       0.45      0.37      0.40      8376
           1       0.56      0.59      0.58      8380

    accuracy                           0.52     25000
   macro avg       0.52      0.52      0.52     25000
weighted avg       0.52      0.52      0.52     25000



## Nerual Networks

In [45]:
from sklearn.neural_network import MLPClassifier
NN_clf_stop = MLPClassifier(random_state=1, max_iter=30, hidden_layer_sizes=(16,16), tol=1e-5, early_stopping=True, learning_rate_init=0.01)
NN_clf_stop.fit(tf_x_train, Y_train)
NN_pred_stop = NN_clf_stop.predict(tf_x_test)
print(classification_report(Y_test, NN_pred_stop))

              precision    recall  f1-score   support

          -1       0.70      0.69      0.69      8244
           0       0.56      0.51      0.53      8376
           1       0.66      0.73      0.70      8380

    accuracy                           0.64     25000
   macro avg       0.64      0.64      0.64     25000
weighted avg       0.64      0.64      0.64     25000



## Support vector machines 

In [47]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
SGD_SVM_clf = make_pipeline(StandardScaler(with_mean=False), SGDClassifier(max_iter=500, tol=1e-3,))
SGD_SVM_clf.fit(tf_x_train, Y_train)
SGD_SVM_pred = SGD_SVM_clf.predict(tf_x_test)
print(classification_report(Y_test, SGD_SVM_pred))

              precision    recall  f1-score   support

          -1       0.66      0.66      0.66      8244
           0       0.54      0.46      0.50      8376
           1       0.62      0.71      0.66      8380

    accuracy                           0.61     25000
   macro avg       0.61      0.61      0.61     25000
weighted avg       0.61      0.61      0.61     25000



## Second Logistic Regression

In [49]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
#Logreg = LogisticRegression(max_iter=500, solver = 'newton-cg', n_jobs=8).fit(x_train_fit,y_train)
Logreg = LogisticRegression(random_state=30, max_iter=10000,solver='newton-cg',C=50).fit(tf_x_train,Y_train)
Logreg_pred = Logreg.predict(tf_x_test)

print(classification_report(Y_test, Logreg_pred))

              precision    recall  f1-score   support

          -1       0.67      0.69      0.68      8244
           0       0.55      0.52      0.53      8376
           1       0.67      0.68      0.68      8380

    accuracy                           0.63     25000
   macro avg       0.63      0.63      0.63     25000
weighted avg       0.63      0.63      0.63     25000



# Data Splitting

In [22]:
X=df.iloc[:,2:]
Y=df["label"]

0        1
1        1
2        1
3        1
4        1
        ..
99994   -1
99995   -1
99996   -1
99997   -1
99998   -1
Name: label, Length: 99999, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

print(x_train.shape,y_train.shape)

(74999, 3681) (74999,)


## Third Logistic Regrsession


In [40]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000,solver='saga')
clf.fit(tf_x_train,Y_train)

LogisticRegression(max_iter=10000, solver='saga')

In [41]:
y_test_pred=clf.predict(tf_x_test)

In [42]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_test_pred))

              precision    recall  f1-score   support

          -1       0.68      0.70      0.69      8244
           0       0.56      0.52      0.54      8376
           1       0.68      0.70      0.69      8380

    accuracy                           0.64     25000
   macro avg       0.64      0.64      0.64     25000
weighted avg       0.64      0.64      0.64     25000

